# Capstone Project - The Battle of the Neighbourhoods
#### Applied Data Science Capstone by IBM/Coursera

### Deciding a suitable location to open an Italian restaurant in New York city

In [49]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

In [50]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [51]:
neighbourhoods_data = newyork_data['features']
neighbourhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [52]:
# define the dataframe columns
column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)

In [53]:
for data in neighbourhoods_data:
    borough = neighbourhood_name = data['properties']['borough'] 
    neighbourhood_name = data['properties']['name']
        
    neighbourhood_latlon = data['geometry']['coordinates']
    neighbourhood_lat = neighbourhood_latlon[1]
    neighbourhood_lon = neighbourhood_latlon[0]
    
    neighbourhoods = neighbourhoods.append({'Borough': borough,
                                          'Neighbourhood': neighbourhood_name,
                                          'Latitude': neighbourhood_lat,
                                          'Longitude': neighbourhood_lon}, ignore_index=True)

In [54]:
neighbourhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [55]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighbourhoods['Borough'].unique()),
        neighbourhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [56]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [57]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [58]:
neighbourhoods['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [59]:
bronx_data = neighbourhoods[neighbourhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Defining Foursquare Credentials and Version

Cell is hidden for security


## 2. Explore Neighborhoods in New York

In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id'],
            v['venue']['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Category Id',
                  'Venue Id']
    
    return(nearby_venues)

#### Code to run the above function on each neighborhood and create a new dataframe called *newyork_venues*.

In [62]:
# type your answer here

newyork_venues = getNearbyVenues(names=bronx_data['Neighbourhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )
newyork_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category Id,Venue Id
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,4bf58dd8d48988d1d0941735,4c537892fd2ea593cb077a28
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy,4bf58dd8d48988d10f951735,4d6af9426107f04dedeb297a
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,4bf58dd8d48988d1c9941735,4c783cef3badb1f7e4244b54
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant,4bf58dd8d48988d144941735,508af256e4b0578944c87392
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop,4bf58dd8d48988d148941735,4c25c212f1272d7f836385c5


In [63]:
print(newyork_venues.shape)

(1239, 9)


In [64]:
newyork_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category Id,Venue Id
Neighbourhood,,,,,,,,
Allerton,27,27,27,27,27,27,27,27
Baychester,24,24,24,24,24,24,24,24
Bedford Park,34,34,34,34,34,34,34,34
Belmont,100,100,100,100,100,100,100,100
Bronxdale,13,13,13,13,13,13,13,13
Castle Hill,9,9,9,9,9,9,9,9
City Island,30,30,30,30,30,30,30,30
Claremont Village,14,14,14,14,14,14,14,14
Clason Point,10,10,10,10,10,10,10,10


In [66]:
print('There are {} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

There are 169 uniques categories.


#### Filtering out the Italian Restaurants

In [67]:
italian_restaurant_category = '4bf58dd8d48988d110941735'  # obtained from Foursquare API

In [68]:
italian = newyork_venues[newyork_venues['Venue Category Id'] == italian_restaurant_category]

In [69]:
italian.head(7)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category Id,Venue Id
130,Woodlawn,40.898273,-73.867315,Patrizia's Of Woodlawn,40.900638,-73.867724,Italian Restaurant,4bf58dd8d48988d110941735,511edb6de4b0d58346fd272d
219,Pelham Parkway,40.857413,-73.854756,Enzo's,40.854232,-73.854362,Italian Restaurant,4bf58dd8d48988d110941735,4bf96ae65317a593d837017f
227,Pelham Parkway,40.857413,-73.854756,Pasta Pasta,40.854788,-73.854929,Italian Restaurant,4bf58dd8d48988d110941735,4b47f069f964a5208c4426e3
243,City Island,40.847247,-73.786488,Artie's Steak and Seafood,40.849542,-73.787317,Italian Restaurant,4bf58dd8d48988d110941735,4514ed4df964a520e5391fe3
640,Throgs Neck,40.815109,-73.816350,Tosca Marquee,40.819222,-73.817601,Italian Restaurant,4bf58dd8d48988d110941735,4d4456d93616b60c953fe3c2
684,Parkchester,40.837938,-73.856003,Lorraine's Bar and Grill,40.834918,-73.854411,Italian Restaurant,4bf58dd8d48988d110941735,5197f6507dd2310725c35f1e
745,Morris Park,40.847549,-73.850402,Patricia's,40.849313,-73.853536,Italian Restaurant,4bf58dd8d48988d110941735,4b10591af964a5205f6e23e3


In [73]:
# create map of Italian restaurants using latitude and longitude values
map_italian = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Venue, neighbourhood in zip(italian['Venue Latitude'], italian['Venue Longitude'], italian['Venue'],italian['Neighbourhood']):
    label = '{}, {}'.format(Venue, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='green',icon='info-sign')
        ).add_to(map_italian) 
    
map_italian

## 3. Analyzing Each Neighborhood

In [74]:
# one hot encoding
newyork_onehot = pd.get_dummies(italian[['Venue']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighbourhood'] = italian['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_onehot

,Neighbourhood,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
130,Woodlawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
219,Pelham Parkway,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
227,Pelham Parkway,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
243,City Island,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
640,Throgs Neck,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
684,Parkchester,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
745,Morris Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
767,Belmont,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
770,Belmont,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
781,Belmont,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
newyork_grouped = newyork_onehot.groupby('Neighbourhood').mean().reset_index()
newyork_grouped

,Neighbourhood,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
0,Belmont,0.055556,0.055556,0.055556,0.055556,0.0,0.0,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.0,0.0,0.055556,0.000000,0.0,0.000000,0.0,0.055556,0.055556,0.055556,0.0,0.0,0.00,0.0,0.0,0.00,0.055556,0.055556,0.00,0.0,0.055556,0.0,0.00,0.055556,0.000000
1,Bronxdale,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.5,0.000000,0.0,0.00,0.000000,0.000000
2,City Island,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
3,Concourse,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,1.0,0.00,0.000000,0.000000
4,Concourse Village,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,1.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
5,Edgewater Park,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.25,0.0,0.0,0.25,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.25,0.000000,0.000000
6,Morris Park,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
7,North Riverdale,0.000000,0.000000,0.000000,0.000000,0.0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
8,Parkchester,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
9,Pelham Bay,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.333333,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.333333


## 4. Cluster Neighborhoods

In [76]:
# set number of clusters
kclusters = 5

newyork_grouped_clustering = newyork_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 4, 0, 2, 0, 1, 0], dtype=int32)

In [77]:
# add clustering labels
newyork_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

newyork_merged = neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = neighbourhoods.join(newyork_grouped.set_index('Neighbourhood'), on='Neighbourhood')

newyork_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
0,Bronx,Wakefield,40.894705,-73.847201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bronx,Co-op City,40.874294,-73.829939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bronx,Eastchester,40.887556,-73.827806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bronx,Riverdale,40.890834,-73.912585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
newyork_merged = newyork_merged.dropna()
newyork_merged.tail()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
39,Bronx,Edgewater Park,40.821986,-73.813885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.25,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.0,0.0
42,Bronx,Pelham Gardens,40.862966,-73.841612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
43,Bronx,Concourse,40.834284,-73.915589,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,1.0,0.00,0.0,0.0
268,Bronx,Concourse Village,40.824780,-73.915847,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
297,Bronx,Bronxdale,40.852723,-73.861726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.5,0.0,0.0,0.00,0.0,0.0


In [79]:
newyork_merged['Cluster Labels'] = newyork_merged['Cluster Labels'].astype(int)

In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighbourhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examinining Clusters

#### Cluster 1

In [81]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
7,Woodlawn,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,1.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
11,Pelham Parkway,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.5,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
12,City Island,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
28,Throgs Neck,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,1.00,0.000000,0.000000
34,Belmont,0.055556,0.055556,0.055556,0.055556,0.0,0.0,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.0,0.0,0.055556,0.000000,0.0,0.000000,0.0,0.055556,0.055556,0.055556,0.0,0.0,0.00,0.0,0.0,0.00,0.055556,0.055556,0.00,0.0,0.055556,0.0,0.00,0.055556,0.000000
36,North Riverdale,0.000000,0.000000,0.000000,0.000000,0.0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
37,Pelham Bay,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.333333,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.333333
39,Edgewater Park,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.25,0.0,0.0,0.25,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.25,0.000000,0.000000
42,Pelham Gardens,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000
297,Bronxdale,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.5,0.000000,0.0,0.00,0.000000,0.000000


#### Cluster 2

In [82]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
30,Parkchester,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Cluster 3

In [83]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
33,Morris Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Cluster 4

In [84]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
43,Concourse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


#### Cluster 5

In [85]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighbourhood,Ann & Tony's - An Original Arthur Avenue Restaurant,Antonio's Trattoria,Arthur Avenue Cafe,Arthur Avenue Fiasco,Artie's Steak and Seafood,Beccofino,Cafe al Mercato,Calabria Pork Store,David Greco's Arthur Avenue Trattoria,Dominick's Restaurant,Emilia's Restaurant,Enzo's,F & J Pine Tavern,Fratelli's,Gerbasi Ristorante,Giovanni's Italian Restaurant,Lorraine's Bar and Grill,Lucca Italian Restaurant,Madison's,Mario's Restaurant & Catering,Michaelangelo's Coal Fired Brick Oven Pizza Restaurant,Pasquale Riggaletto's,Pasta Pasta,Patricia's,Patricia's of Tremont,Patrizia's Of Woodlawn,Porto Salvo,Rino's Italian Restaurant,Roberto,San Gennaro Ristorante,Spoto's Italian Restaurant,The Pines,Tino's Delicatessen,Tiny's Pizza & Pasta,Tosca Marquee,Tra Di Noi,Villa Barone Ristorante
268,Concourse Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
